In [ ]:
!pip install openai
from openai import OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [ ]:
import os
import sys
sys.path.append('/content/drive/My Drive/CS 159')

from utils import extract_answer, evaluate_file, evaluate_across_files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
q2 = "You have been given a circular path consisting of 12 connected dots. At the start, you are positioned on the dot that is located at the top of the path, where you find a crash helmet. Moving in a clockwise direction from the crash helmet, the elements on the path are a trimaran, a sleeping bag, a dhole, a sulphur butterfly, a stove, a rotisserie, a stupa, a bagel, a gibbon, a fountain, and a Geoffroy's spider monkey. Starting from the rotisserie, you move around the ring by 11 steps in a counter-clockwise direction, and you move around the ring by 4 steps in a clockwise direction, and you move around the ring by 11 steps in a counter-clockwise direction, and you move around the ring by 7 steps in a counter-clockwise direction, and you move around the ring by 2 steps in a clockwise direction. What will you find?"

In [ ]:
import re
import glob
import requests
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from tqdm.notebook import tqdm


data_path = "/content/drive/MyDrive/CS159/data/raw/SpatialEvalLLM/"

In [ ]:
class GPT4Client():
    def __init__(self, api_key, temper = 0.7):
        self.client = OpenAI(api_key=api_key)
        self.temper = temper

    def predict(self, prompt):
        result = self.client.chat.completions.create(
            messages=[
                {'role': 'user', 'content': prompt},
            ],
            model="gpt-4-turbo",
            temperature = self.temper
        )
        return result.choices[0].message.content

class GPT3Client():
    def __init__(self, api_key, temper = 0.7):
        self.client = OpenAI(api_key=api_key)
        self.temper = temper

    def predict(self, prompt):
        result = self.client.chat.completions.create(
            messages=[
                {'role': 'user', 'content': prompt},
            ],
            model="gpt-3.5-turbo",
            temperature = self.temper
        )
        return result.choices[0].message.content


gpt4 = GPT4Client('--redacted--')
gpt3 = GPT3Client('--redacted--')


In [ ]:
import glob
import pandas as pd
import re
from tqdm.notebook import tqdm
from pathlib import Path

def process_model_evaluation(model_name, model_instance, data_path, leading_prompt, trailing_prompt, suffix):
    print(f"Starting evaluation for {model_name} with trailing prompt: {trailing_prompt[:30]}...")

    scores_df = evaluate_across_files(
        model=model_instance,
        leading_prompt=leading_prompt,
        trailing_prompt=trailing_prompt,
        data_path=data_path,
        suffix=suffix,
        use_code=False,
        save=True
    )

    save_path = f"/content/drive/MyDrive/CS 159/{model_name}_evaluation_{suffix}.csv"

    scores_df.to_csv(save_path, index=False)
    print(f"Results saved to {save_path}")

# Models dictionary
models = {
    "GPT4": gpt4
    #"GPT3": gpt3
}

leading_prompt = ""
suffixes = ["zero_shot_vot"]

trailing_prompts = [
     "\n\nVisualize the state of the map after each step. Place the answer, in lower case, with asterisks before and after like *this*. No asterisks anywhere else in the response."
]

for model_name, model_instance in models.items():
    for trailing_prompt, suffix in zip(trailing_prompts, suffixes):
        process_model_evaluation(model_name, model_instance, data_path, leading_prompt, trailing_prompt, suffix)

print("All evaluations are complete.")


Starting evaluation for GPT4 with trailing prompt: 

Visualize the state of the m...
Data path: /content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/
Map global path: /content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/map_global/*
Map local path: /content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/map_local/*
Found 9 global files
Found 5 local files
Files to process: ['/content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/map_global/type-ring_size-12_steps-8_seed-12_n-100.jsonl', '/content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/map_global/type-ring_size-9_steps-4_seed-9_n-100.jsonl', '/content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/map_global/type-square_size-3_steps-4_seed-3_n-100.jsonl', '/content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/map_global/type-square_size-3_steps-8_seed-3_n-100.jsonl', '/content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/map_global/type-square_size-3_steps-8_seed-3_n-100_special_order-random_order.jsonl', '/content/drive/MyDrive/CS 1

Evaluating files:   0%|          | 0/14 [00:00<?, ?it/s]

Intermediate results saved to /content/drive/MyDrive/CS 159/GPT4Client/type-ring_size-12_steps-8_seed-12_n-100_evaluation_no_code_zero_shot_vot.csv
Intermediate results saved to /content/drive/MyDrive/CS 159/GPT4Client/type-ring_size-9_steps-4_seed-9_n-100_evaluation_no_code_zero_shot_vot.csv
Intermediate results saved to /content/drive/MyDrive/CS 159/GPT4Client/type-square_size-3_steps-4_seed-3_n-100_evaluation_no_code_zero_shot_vot.csv
Intermediate results saved to /content/drive/MyDrive/CS 159/GPT4Client/type-square_size-3_steps-8_seed-3_n-100_evaluation_no_code_zero_shot_vot.csv
Intermediate results saved to /content/drive/MyDrive/CS 159/GPT4Client/type-square_size-3_steps-8_seed-3_n-100_special_order-random_order_evaluation_no_code_zero_shot_vot.csv
Intermediate results saved to /content/drive/MyDrive/CS 159/GPT4Client/type-square_size-3_steps-8_seed-3_n-100_special_order-snake_order_evaluation_no_code_zero_shot_vot.csv
Intermediate results saved to /content/drive/MyDrive/CS 159/G

In [ ]:
directory = '/content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM'
data_path = '/content/drive/MyDrive/CS 159/data/raw/SpatialEvalLLM/'
try:
    files = os.listdir(directory)
except FileNotFoundError:
    files = ["Directory not found"]

files

['map_local', '.git', '.gitattributes', 'README.md', 'map_global']

In [ ]:
# internal representations
models = {
    "GPT4": gpt4,
    "GPT3": gpt3,
}
leading_prompt = ""

prompts = {
    'base': '\nPresent reasoning step-by-step. Place the answer, in lower case, with asterisks before and after like *this*. No asterisks anywhere else in the response.',
    'grid': '''\nPresent reasoning step-by-step.\nAt each step, represent the current state (including your current location) as
+-----------+-----------+---------+
| Object 1 | Object 2 | Object 3 |
+-----------+-----------+-----------+
| Object 4 | Object 5* | Object 6 |
+----------+-----------+-----------+
| Object 7 | Object 8 | Object 9 |
+----------+-----------+----------+
where the asterisk * corresponds to the current location.
Place the answer, in lower case, with asterisks before and after like *this*. No asterisks anywhere else in the response.''',
    'csv': '''\nPresent reasoning step-by-step.\nAt each step, represent the current state (including your current location) as
Object 1, Object 2, Object 3
Object 4, Object 5*, Object 6
Object 7, Object 8, Object 9
where the asterisk * corresponds to the current location.
Place the answer, in lower case, with asterisks before and after like *this*. No asterisks anywhere else in the response.''',
    'coord': '''\nPresent reasoning step-by-step.\nAt each step, represent the current state (including your current location) as
(1, 1): Object 1, (1, 2): Object 2, (1, 3): Object 3, (2, 1): Object 4, (2, 2): Object 5*, (2, 3): Object 6, (3, 1): Object 7, (3, 2): Object 8, (3, 3): Object 9
where the asterisk * corresponds to the current location.
Place the answer, in lower case, with asterisks before and after like *this*. No asterisks anywhere else in the response.''',
    'colbycol': '''\nPresent reasoning step-by-step.\nAt each step, represent the current state (including your current location) as
Object 1, Object 4, Object 7
Object 2, Object 5*, Object 8
Object 3, Object 6, Object 9
where the asterisk * corresponds to the current location.
Place the answer, in lower case, with asterisks before and after like *this*. No asterisks anywhere else in the response.''',
}
# trailing_prompt = "\n\nVisualize the state of the map after each step.\n\nVisualize the state after each step.\n\nPlace the answer, in lower case, with asterisks before and after like *this*. No asterisks anywhere else in the response."

for model_name, model_instance in models.items():
    for label, prompt in prompts.items():
        save_path = f'/content/drive/MyDrive/CS159/internal_representations/type-square_size-3_steps-4_seed-3_n-100_{model_name}_' + label + '.csv'
        if os.path.isfile(save_path):
          print(f"Existing file found at path. Continuing...")
          continue
        print('Running', save_path)
        # Run the evaluation for the specific configuration
        # process_model_evaluation(model_name, model_instance, data_path, leading_prompt, trailing_prompt, suffix)
        df = evaluate_file(
            data_path + 'map_global/type-square_size-3_steps-4_seed-3_n-100.jsonl',
            model_instance,
            leading_prompt,
            prompt,
            use_code=False,
            suffix=label,
            pbar=tqdm(),
            debug=False
        )
        df.to_csv(save_path)


Existing file found at path. Continuing...
Existing file found at path. Continuing...
Existing file found at path. Continuing...
Existing file found at path. Continuing...
Running /content/drive/MyDrive/CS159/internal_representations/type-square_size-3_steps-4_seed-3_n-100_GPT4_colbycol.csv


0it [00:00, ?it/s]

Intermediate results saved to /content/drive/MyDrive/CS 159/GPT4Client/type-square_size-3_steps-4_seed-3_n-100_evaluation_no_code_colbycol.csv
Running /content/drive/MyDrive/CS159/internal_representations/type-square_size-3_steps-4_seed-3_n-100_GPT3_base.csv


0it [00:00, ?it/s]

Intermediate results saved to /content/drive/MyDrive/CS 159/GPT3Client/type-square_size-3_steps-4_seed-3_n-100_evaluation_no_code_base.csv
Running /content/drive/MyDrive/CS159/internal_representations/type-square_size-3_steps-4_seed-3_n-100_GPT3_grid.csv


...

Intermediate results saved to /content/drive/MyDrive/CS 159/GPT3Client/type-square_size-3_steps-4_seed-3_n-100_evaluation_no_code_csv.csv
Running /content/drive/MyDrive/CS159/internal_representations/type-square_size-3_steps-4_seed-3_n-100_GPT3_coord.csv


0it [00:00, ?it/s]

Intermediate results saved to /content/drive/MyDrive/CS 159/GPT3Client/type-square_size-3_steps-4_seed-3_n-100_evaluation_no_code_coord.csv
Running /content/drive/MyDrive/CS159/internal_representations/type-square_size-3_steps-4_seed-3_n-100_GPT3_colbycol.csv


0it [00:00, ?it/s]

Intermediate results saved to /content/drive/MyDrive/CS 159/GPT3Client/type-square_size-3_steps-4_seed-3_n-100_evaluation_no_code_colbycol.csv


In [ ]:
import pandas as pd
import glob

# Load the results from the files
result_files = glob.glob("/content/drive/MyDrive/CS 159/*.csv")

# Initialize a dictionary to hold the aggregated results
aggregated_results = {}

for file in result_files:
    # Extract the model name and code mode from the filename
    filename = file.split("/")[-1]
    model_name = filename.split("_evaluation_")[0]
    file_mode = filename.split("_evaluation_")[1].split(".csv")[0]

    # Load the CSV file
    df = pd.read_csv(file)

    if model_name not in aggregated_results:
        aggregated_results[model_name] = {"code": {"Correct": 0, "Total": 0},
                                          "no_code": {"Correct": 0, "Total": 0}}

    # Sum up the correct and total counts for each mode
    aggregated_results[model_name][file_mode]["Correct"] += df["Correct"].sum()
    aggregated_results[model_name][file_mode]["Total"] += df["Total"].sum()

# Create a DataFrame to display the results
summary_data = []
for model, results in aggregated_results.items():
    for mode in ["code", "no_code"]:
        accuracy = results[mode]["Correct"] / results[mode]["Total"] if results[mode]["Total"] != 0 else 0
        summary_data.append({"Model": model, "Mode": mode, "Accuracy": accuracy})

summary_df = pd.DataFrame(summary_data)

# Display the aggregated results
display(summary_df)
